In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
# from keras.regularizers import l2
# from keras.regularizers import l1
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from tensorflow import keras

In [2]:
ruta_archivo_train = f'trainReg.txt'
ruta_archivo_test = f'testReg.txt'

#scaler = StandardScaler()

# Ajustar el escalador en los datos de entrenamiento y transformar los datos
#train_tot = scaler.fit_transform(pd.read_csv(ruta_archivo_train, delimiter=','))
#test_tot = scaler.fit_transform(pd.read_csv(ruta_archivo_test, delimiter=','))
train_tot = pd.read_csv(ruta_archivo_train, delimiter=',')

test_tot = pd.read_csv(ruta_archivo_test, delimiter=',')

# print(train_tot.head())
# print(test_tot.head())


x_train_ori = train_tot[train_tot.columns[~train_tot.columns.isin(["V1", "V4"])]]
y_train_ori =train_tot["V1"]
x_test_ori =test_tot[test_tot.columns[~test_tot.columns.isin(["Id", "V4"])]]

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x_train_ori, y_train_ori, test_size=0.3)

In [4]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((54445, 89), (23334, 89), (54445,), (23334,))

In [5]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54445 entries, 11149 to 1419
Data columns (total 89 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V2      54445 non-null  float64
 1   V3      54445 non-null  float64
 2   V5      54445 non-null  float64
 3   V6      54445 non-null  float64
 4   V7      54445 non-null  float64
 5   V8      54445 non-null  float64
 6   V9      54445 non-null  float64
 7   V10     54445 non-null  float64
 8   V11     54445 non-null  float64
 9   V12     54445 non-null  float64
 10  V13     54445 non-null  float64
 11  V14     54445 non-null  float64
 12  V15     54445 non-null  float64
 13  V16     54445 non-null  float64
 14  V17     54445 non-null  float64
 15  V18     54445 non-null  float64
 16  V19     54445 non-null  float64
 17  V20     54445 non-null  float64
 18  V21     54445 non-null  float64
 19  V22     54445 non-null  float64
 20  V23     54445 non-null  float64
 21  V24     54445 non-null  float64
 22  

In [19]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scale = scaler.transform(x_train)
x_test_scale = scaler.transform(x_test)
x_test_scale_ori = scaler.transform(x_test_ori)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_spar

In [20]:
# Aplicar PCA a los datos de entrenamiento y prueba con 23 componentes
n_components = 89

pca = PCA(n_components=n_components)
pca.fit(x_train_scale)

x_train_pca = pca.transform(x_train_scale)
x_test_pca = pca.transform(x_test_scale)
x_test_pca_ori = pca.transform(x_test_scale_ori)
# Convert NumPy arrays to Pandas DataFrames
columns = [f"PC{i}" for i in range(1, n_components + 1)]
x_train_pca_df = pd.DataFrame(x_train_pca, columns=columns)
x_test_pca_df = pd.DataFrame(x_test_pca, columns=columns)
x_test_pca_ori_df = pd.DataFrame(x_test_pca_ori, columns=columns)

In [8]:
x_train_pca_df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89
0,-1.800417,-1.048593,-2.462892,-1.903880,-2.706123,0.126593,0.112773,-1.088876,0.519053,0.949751,...,0.046102,-0.191445,0.254158,-0.171608,-0.041422,-0.052408,0.068220,0.151329,-0.018917,0.164452
1,0.531006,-1.571477,-4.191237,1.132363,-3.072745,-0.844856,3.226900,-0.369838,-0.544767,0.127943,...,0.569895,-0.934294,-0.097634,0.413375,0.239784,0.509209,-0.053643,0.017514,0.397854,0.353449
2,-2.635572,-2.491409,-2.623165,4.307601,1.001659,-0.501623,0.094527,0.432506,-0.199316,-1.306413,...,-0.769221,0.142313,-0.612615,0.041553,0.090951,0.189377,-0.573533,0.210116,0.176411,0.005857
3,-2.244709,0.009989,1.520027,2.107475,-0.679656,-0.993198,-1.124388,1.164868,-0.760511,2.800769,...,-0.340773,-0.034804,0.014936,0.026822,-0.216965,0.243134,-0.081667,0.011710,-0.198955,-0.351908
4,2.292275,0.616649,-0.609644,-2.238110,-0.776298,0.663033,-0.684003,-1.915263,0.214779,-0.211680,...,-0.391347,-0.200085,0.256393,0.291201,-0.153328,-0.116602,0.224596,-0.063850,0.184580,-0.098347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54440,-3.731527,-1.702200,1.968426,0.728472,-0.327639,-0.158400,1.379059,-0.232182,-0.210272,0.227191,...,0.026098,0.204531,0.316552,-0.198237,-0.097420,0.132575,0.189118,-0.191142,0.125224,-0.027779
54441,-4.232238,-1.034800,1.247961,0.670156,0.064895,0.137860,-0.387643,0.028448,-0.216407,-1.261008,...,-0.248311,-0.031598,0.272810,0.003516,-0.057352,0.098590,-0.066298,-0.053715,-0.125487,0.114177
54442,2.718412,6.907340,-3.213658,-1.794978,-0.014764,-1.699097,-0.993768,-1.926511,0.008270,-1.773467,...,0.083142,0.217633,-0.444122,0.401213,0.391834,0.323249,0.301229,-0.293882,-0.062669,-0.139916
54443,-0.849227,-3.362063,-4.446842,0.903412,-1.740605,-1.255641,1.484005,1.689847,1.033102,-0.185772,...,-0.248262,-0.487867,0.444995,0.016783,0.013068,0.732499,-0.425857,0.163947,0.120960,0.404139


In [21]:
# Create a Gradient Boosting Regressor model
model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    max_features='sqrt',
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=3,
)
# Train the model
model.fit(x_train_pca_df, y_train)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_leaf=3,
                      random_state=42)

In [22]:

y_pred_train = model.predict(x_train_pca_df)
y_pred_test = model.predict(x_test_pca_df)
mse_train = mean_squared_error(y_train, y_pred_train)
print("mse")
print(mse_train)
mse_test = mean_squared_error(y_test, y_pred_test)
print("mse")
print(mse_test)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_spar

mse
24.678082693277272
mse
90.11659941269727


In [23]:
y_pred_test_ori = model.predict(x_test_pca_ori_df)

c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\SGM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [24]:
# Create the results DataFrame
results = pd.DataFrame({"Id": test_tot["Id"], "y": y_pred_test_ori.flatten()})

# Print the first few rows of the results DataFrame
print(results.head())

# Guardar el DataFrame en un archivo CSV
# ruta_resultados = r'C:\Users\Usuario\Downloads\Statistical\Statistical\proy\Regre\boost11100.csv'

   Id            y
0   1  2003.900833
1   2  1997.359920
2   3  2000.168710
3   4  2002.754024
4   5  2004.414182


In [25]:
ruta_resultados = f'resultados_rf_100.csv'

results.to_csv(ruta_resultados, index=False)

In [5]:
df = results.copy()

In [ ]:
df['y_new'] = df['y']

In [21]:
df['y_1'] = df['y_new'].round()
df['y'] = df['y_new_new'].round()

In [22]:
df

,Id,y,y_new,y_new_new,y_1
0,1,2006.0,2006.405057,2006.005057,2006.0
1,2,1998.0,1998.716301,1998.316301,1999.0
2,3,2006.0,2005.587338,2005.987338,2006.0
3,4,2002.0,2002.052711,2001.652711,2002.0
4,5,2005.0,2004.248900,2004.648900,2004.0
...,...,...,...,...,...
18652,18653,1997.0,1997.836605,1997.436605,1998.0
18653,18654,2002.0,2002.856300,2002.456300,2003.0
18654,18655,2000.0,1999.269548,1999.669548,1999.0
18655,18656,2001.0,2000.907672,2000.507672,2001.0


In [23]:
ruta_resultados = f'y_test.csv'

df[['Id', 'y']].to_csv(ruta_resultados, index=False)

In [18]:
random_changes = np.random.choice([-0.4, 0.4], size=len(df['y_new']))

In [19]:
df['y_new_new'] = df['y_new'] + random_changes

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt']
}

# Paso 4: Crear el modelo de RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)

# Paso 5: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid, 
    scoring='neg_mean_squared_error', 
    cv=5,
    n_jobs=-1,
    verbose=3
)

# Paso 6: Ajustar el modelo utilizando GridSearchCV
grid_search.fit(x_train_pca_df, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


KeyboardInterrupt: 